In [1]:
%%script C:\Users\Jan Catherine\AppData\Local\Programs\Python\Python38\python.exe
import sys
print(sys.version)

3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [2]:
pwd

'C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\MachineLearning'

In [3]:
import os
os.chdir('../../Data_Collection_02/02_Preprocessing/04_ML')

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

import matplotlib.pyplot as plt
import pandas as pd
dataset = pd.read_csv('02_ML_Train.csv',  delimiter=',')
testdata = pd.read_csv('02_ML_Test.csv',  delimiter=',')

In [5]:
#os.chdir('../../../Data_Collection_03/02_Preprocessing/04_ML')
#testdata = pd.read_csv('02_ML_Dataset.csv',  delimiter=',')

In [6]:
#ext
ordinal_label = {k: i for i, k in enumerate(dataset['ext'].unique(), 0)}
dataset['ext'] = dataset['ext'].map(ordinal_label)

#domDigitStart
dataset['domDigitStart']=np.where(dataset['domDigitStart']==True,1,0)

#hasHttp
dataset['hasHttp']=np.where(dataset['hasHttp']==True,1,0)

#hasHttps
#dataset['hasHttps']=np.where(dataset['hasHttps']==True,1,0)

#has_ip
#dataset['has_ip']=np.where(dataset['has_ip']==True,1,0)

#word_blog
#dataset['word_blog']=np.where(dataset['word_blog']==True,1,0)

#word_socmed
dataset['word_socmed']=np.where(dataset['word_socmed']==True,1,0)

#word_news
dataset['word_news']=np.where(dataset['word_news']==True,1,0)

#word_covid
#dataset['word_covid']=np.where(dataset['word_covid']==True,1,0)


In [7]:
#ext
ordinal_label = {k: i for i, k in enumerate(testdata['ext'].unique(), 0)}
testdata['ext'] = testdata['ext'].map(ordinal_label)

#domDigitStart
testdata['domDigitStart']=np.where(testdata['domDigitStart']==True,1,0)

#hasHttp
testdata['hasHttp']=np.where(testdata['hasHttp']==True,1,0)

#hasHttps
#testdata['hasHttps']=np.where(testdata['hasHttps']==True,1,0)

#has_ip
#testdata['has_ip']=np.where(testdata['has_ip']==True,1,0)

#word_blog
#testdata['word_blog']=np.where(testdata['word_blog']==True,1,0)

#word_socmed
testdata['word_socmed']=np.where(testdata['word_socmed']==True,1,0)

#word_news
testdata['word_news']=np.where(testdata['word_news']==True,1,0)

#word_covid
#testdata['word_covid']=np.where(testdata['word_covid']==True,1,0)

In [8]:
dataset.head()

,0,0002100µm,1,12,01lm,2,3,033nm,4,5,...,numUnderscore,numEqual,hasHttp,ext,num_%20,num_@,registered.1,word_socmed,word_news,Fake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0.0,0,0.228571,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1,0.0,0,0.314286,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1,0.0,0,0.514286,0,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,0.0,0,0.800000,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1,0.0,0,0.257143,0,1,0


In [9]:
X = dataset.drop("Fake", axis=1)
y = dataset["Fake"]
X.shape

(6451, 9613)

In [10]:
y.shape

(6451,)

In [11]:
Xtest = testdata.drop("Fake", axis=1)
ytest = testdata["Fake"]
Xtest.shape

(1613, 9613)

In [12]:
ytest.shape

(1613,)

In [13]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
X_train = X
X_test = Xtest
y_train = y
y_test = ytest

In [14]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [15]:
def score_model(model, params, cv):

    smoter = SMOTE(random_state=42)
    
    pscores = []
    ascores = []
    rscores = []
    fscores = []

    for train_fold_index, val_fold_index in cv.split(X, y):
        X_train_fold, y_train_fold = X.iloc[train_fold_index], y.iloc[train_fold_index]
        X_val_fold, y_val_fold = X.iloc[val_fold_index], y.iloc[val_fold_index]

        X_train_fold_upsample, y_train_fold_upsample = smoter.fit_resample(X_train_fold,
                                                                           y_train_fold)
        model_obj = model(**params).fit(X_train_fold_upsample, y_train_fold_upsample)

        precisionscore = precision_score(y_val_fold, model_obj.predict(X_val_fold))
        accuracyscore = accuracy_score(y_val_fold, model_obj.predict(X_val_fold))
        recallscore = recall_score(y_val_fold, model_obj.predict(X_val_fold))
        f1score = f1_score(y_val_fold, model_obj.predict(X_val_fold))
        pscores.append(precisionscore)
        ascores.append(accuracyscore)
        rscores.append(recallscore)
        fscores.append(f1score)
        
    scores = [pscores, ascores, rscores, fscores]
    
    precisionscore2 = precision_score(y_test, model_obj.predict(X_test))
    accuracyscore2 = accuracy_score(y_test, model_obj.predict(X_test))
    recallscore2 = recall_score(y_test, model_obj.predict(X_test))
    f1score2 = f1_score(y_test, model_obj.predict(X_test))
    
    scores2 = [precisionscore2, accuracyscore2, recallscore2, f1score2]
    scores_1 = np.array(scores)
    scores_2 = np.array(scores2)
    return scores_1, scores_2

In [16]:
def score_mean(scores):
    
    prec_m = (np.mean(scores[0]))
    acc_m = (np.mean(scores[1]))
    recall_m = (np.mean(scores[2]))
    f1_m = (np.mean(scores[3]))     
    results =np.asarray([prec_m, acc_m, recall_m, f1_m])
    
    return np.array(results)

In [17]:
""""
#RANDOM FOREST
rfparams ={}
scores=score_model(RandomForestClassifier, rfparams, cv=kf)
print(scores)
"""

'"\n#RANDOM FOREST\nrfparams ={}\nscores=score_model(RandomForestClassifier, rfparams, cv=kf)\nprint(scores)\n'

In [18]:
"""
scoremean = score_mean(scores)
print(scoremean)
"""

'\nscoremean = score_mean(scores)\nprint(scoremean)\n'

In [19]:
#LOGISTIC REGRESSION
lrparams ={}
scores, scores2=score_model(LogisticRegression, lrparams, cv=kf)
print(scores)
print(scores2)

C:\Users\Jan Catherine\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Jan Catherine\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

[[0.55232558 0.50318471 0.55479452 0.54404145 0.39779006]
 [0.89775368 0.89767442 0.89922481 0.90310078 0.87829457]
 [0.63333333 0.59398496 0.55479452 0.73943662 0.6       ]
 [0.59006211 0.54482759 0.55479452 0.62686567 0.47840532]]
[0.55785124 0.9032858  0.73369565 0.63380282]


In [20]:
scoremean = score_mean(scores)
print(scoremean)

[0.51042726 0.89520965 0.62430989 0.55899104]


In [21]:
#NAIVE BAYES
nbparams ={}
scores, scores2=score_model(GaussianNB, nbparams, cv=kf)
print(scores)
print(scores2)

[[0.57142857 0.66216216 0.75       0.63636364 0.56578947]
 [0.8954299  0.91550388 0.91317829 0.90852713 0.91472868]
 [0.4        0.36842105 0.34931507 0.3943662  0.35833333]
 [0.47058824 0.47342995 0.47663551 0.48695652 0.43877551]]
[0.65891473 0.91134532 0.46195652 0.54313099]


In [22]:
scoremean = score_mean(scores)
print(scoremean)

[0.63714877 0.90947358 0.37408713 0.46927715]


In [23]:
"""
#SGD
sgdparams ={}
scores=score_model(linear_model.SGDClassifier, sgdparams, cv=kf)
print(scores)
"""

'\n#SGD\nsgdparams ={}\nscores=score_model(linear_model.SGDClassifier, sgdparams, cv=kf)\nprint(scores)\n'

In [24]:
"""
scoremean = score_mean(scores)
print(scoremean)
"""

'\nscoremean = score_mean(scores)\nprint(scoremean)\n'

In [25]:
#SVM
svmparams ={'kernel':'linear'}
scores, scores2=score_model(svm.SVC, svmparams, cv=kf)
print(scores)
print(scores2)

[[0.6984127  0.59482759 0.74285714 0.7109375  0.59663866]
 [0.92254067 0.91395349 0.92635659 0.93178295 0.9248062 ]
 [0.58666667 0.51879699 0.53424658 0.64084507 0.59166667]
 [0.63768116 0.55421687 0.62151394 0.67407407 0.59414226]]
[0.65326633 0.92374458 0.70652174 0.67885117]


In [26]:
scoremean = score_mean(scores)
print(scoremean)

[0.66873472 0.92388798 0.57444439 0.61632566]


In [27]:
import sklearn
print(sklearn.__version__)

1.0.2
